Reference:
Race simulator for Matlab developed by AJK Phillips, 2014.
Credit to AJK Phillips and http://f1metrics.wordpress.com/

The GitHub link below is from another person who uploaded the code by Phillips, who used to put his MATLAB code on Google Drive. (For more details, check the link below)

https://github.com/syllogismos/F1Sim

# RaceSim

For the simulation to run, we need to define the track variables and the driver variables, and import the data we get from DataAnalysis (i.e. the data from Free Practice(s) and Qualifying) to initialize these variables.



## Track Variables

Track variables include:

- Name
- Number of Laps
- Lap Length
- Race Length
- Tyre variables (provided by Pirelli) 
    - Traction
    - Braking
    - Lateral
    - Tyre Stress
    - Track Evolution
    - Asphalt Grip
    - Asphalt Abrasion
    - Downforce

- Fuel Gain (refer to external sources)
- DNF Factor (The likelihood of retirements as race goes on, this value is compounded by laps elapsed (?))
- Safety Car (SC) variables (Could be extended to VSC)
    - Probability of SC
    - Number of Laps per SC period
    - Laptime of SC
    - Closest time gap between cars during the safety car period
    - Pit Window (Laps for pit window under safety car, i.e. Safety Car Window)
    
- Pit variables
    - In lap time loss
    - Out lap time loss
    
- Racing variables
    - Time gained per lap due to DRS
    - Additional tyre wear (in laps) per lap due to DRS
    - Overtaking variables
        - Distance (in seconds) which the car can follow the closest
        - Time loss (in seconds) due to being overtaken. Does not account for resistance (e.g. blocking or slowing down intentionally)
        - Overtaking Threshold (in seconds), i.e. the time needed to attempt to overtake
        
- Start Time standard deviation (???)

- Time difference between fresh tyre compounds
- Relative wear of different compounds
- Wear factor for track (Can be pulled from Pirelli's infographics)

## Driver Variables

Driver variables include:

- name
- Pit Strategy (Predefined, optional strategy)
- Lap Time Standard Deviation, i.e. the variability of their laptimes
- (For plotting) Driver Team Color

## Race Variables

Race variables include:

- Starting Grid Order
- Teammate pairing (Could be considered as a Driver variable)

#### FP2 and Qualifying Teammates Head-to-Head

In [1]:
import fastf1 as ff1
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
from race import *
from qualifying import *
from constants import *
from practice import *
from scipy import stats

# Enable the cache by providing the name of the cache folder
ff1.Cache.enable_cache('cache')

%store -r fp2_race_sim
%store -r times_df

In [48]:
TEAMMATE_MAX_GAP_SECOND = 2
QUALIFYING_MAX_GAP_PERCENTAGE = 1.05
FP2_MAX_GAP_PERCENTAGE = 1.03

QUALIFYING_MIN_LAPTIME = times_df['Fastest Lap'].min()

FP2_MIN_MEDIAN_LAPTIME = fp2_race_sim['MeanLapTime'].median()

In [52]:


teammates_h2h = pd.DataFrame(columns = ['Driver', 'Better FP2', 'Better Qualifying', 'Teammate',  'FP2 Mean LapTime',  'Teammate FP2 Mean LapTime', 'Qualifying LapTime', 'Teammate Qualifying LapTime',  'Qualifying Position','Teammate Qualifying Position'])
teammates_h2h

# To be inserted into teammates_h2h
for driver in TEAMMATE_PAIRS_DICT.keys():
    teammate = TEAMMATE_PAIRS_DICT[driver]
    
    # ".values[0]" extracts the values from the series/dataframe and index 0 extracts the first value, which should be the sole value.
    
    # FP2
    fp2_mean_laptime = fp2_race_sim.loc[fp2_race_sim['Driver'] == driver, 'MeanLapTime'].values[0]
    teammate_fp2_mean_laptime = fp2_race_sim.loc[fp2_race_sim['Driver'] == teammate, 'MeanLapTime'].values[0]
    
    # Qualifying
    q_fastest_lap = times_df.loc[times_df['Abbreviation'] == driver, 'Fastest Lap'].values[0]
    teammate_q_fastest_lap = times_df.loc[times_df['Abbreviation'] == teammate, 'Fastest Lap'].values[0]
    q_position = times_df.index.get_loc(times_df[times_df['Abbreviation'] == driver].index[0]) + 1
    teammate_q_position = times_df.index.get_loc(times_df[times_df['Abbreviation'] == teammate].index[0]) + 1
    
    # Booleans of whether driver has a good qualifying and fp2 sim compared to his teammate
    good_qualifying = q_fastest_lap - teammate_q_fastest_lap < TEAMMATE_MAX_GAP_SECOND \
            and q_fastest_lap < QUALIFYING_MIN_LAPTIME * QUALIFYING_MAX_GAP_PERCENTAGE
    
    good_fp2 = fp2_mean_laptime - teammate_fp2_mean_laptime < TEAMMATE_MAX_GAP_SECOND \
            and fp2_mean_laptime < FP2_MIN_MEDIAN_LAPTIME * FP2_MAX_GAP_PERCENTAGE
    
    teammates_h2h.loc[len(teammates_h2h)] = [driver, good_fp2, good_qualifying, teammate, 
                                             fp2_mean_laptime, teammate_fp2_mean_laptime, 
                                             q_fastest_lap, teammate_q_fastest_lap, 
                                             q_position, teammate_q_position]
    
teammates_h2h

,Driver,Better FP2,Better Qualifying,Teammate,FP2 Mean LapTime,Teammate FP2 Mean LapTime,Qualifying LapTime,Teammate Qualifying LapTime,Qualifying Position,Teammate Qualifying Position
0,RUS,False,True,HAM,85.314800,80.853000,77.377,78.035,1,7
1,VER,True,True,PER,84.173000,84.781500,77.703,78.516,10,11
2,MSC,True,True,MAG,86.678000,85.827600,79.164,78.825,15,13
3,BOT,True,True,ZHO,83.834000,85.335375,78.157,78.573,8,12
4,RIC,True,True,NOR,84.187333,83.917429,78.198,77.769,9,4
5,STR,True,True,VET,84.797700,84.585000,79.137,79.273,14,18
6,ALB,True,True,LAT,85.978000,85.991846,79.256,79.570,17,20
7,TSU,True,True,GAS,85.502333,85.301875,79.240,79.527,16,19
8,OCO,True,True,ALO,84.055000,85.103400,78.018,77.904,5,6
9,SAI,True,True,LEC,83.624000,83.698286,77.421,77.567,2,3
